# Prise en main du code de l'article NGCF


La prise en main du code du site compagnon n'est pas aisé car le code enchaine de multiples fonctions et méthodes.
L'objectif de ce notebook est donc de disséquer le code du site compagnon afin d'en comprendre les rouages et de revenir aux fondements théoriques de l'article.
( pour une lecture avec un plan, utiliser jupyter lab)

Cela permet également de voir comment réécrire le code en tensorflow 2.x.
Dans les points TF non compatibles ont a le placeholder, l'initialisation de Glorot, random_uniform, sparse_retain, tf.div, tf.sparse_dense_matmul





In [99]:
import numpy as np
import tensorflow as tf

from utility.load_data import *


A des fins exploratoires les attributs de l'objet NGCF sont passés en paramètres.

Pour faciliter la lecture on explore le code qu'avec une profondeur de 2 couches.

In [145]:
#donnees ml100
n_users = 943
n_items = 1682
emb_dim = 40 #hyperparamètre
weight_size = [64,64]  #('--layer_size', nargs='?', default='[64]', help='Output sizes of every layer')
n_layers = len(weight_size)  #[64] -> 1 couche.  [64, 64] -> 2 couches
batch_size = 50  # on prend une valeur différente de 64 pour bien distinguer les paramètres
decay = 0.1
mess_dropout =  [0.1,0.1]
#dans un batch on va avoir des items qui seront à la fois positifs et négatifs pour les 2 listes

n_fold = 100
keep_prob = 0.7
node_dropout = [0.7]

In [146]:
from os import getcwd
getcwd()

'/home/jovyan/MonDossier/neural_graph_collaborative_filtering/prise_en_main'

## Création des poids

In [171]:

# A verifier mais les poids MLP ne sont que pour le modèle GMC

def _init_weights(emb_dim, weight_size, n_layers):
        all_weights = dict()

        initializer = tf.keras.initializers.GlorotUniform()
        all_weights['user_embedding'] = tf.Variable(initializer([n_users, emb_dim]), name='user_embedding')
        all_weights['item_embedding'] = tf.Variable(initializer([n_items, emb_dim]), name='item_embedding')
        
        weight_size_list = [emb_dim] + weight_size

        for k in range(n_layers):
            all_weights['W_gc_%d' %k] = tf.Variable(
                initializer([weight_size_list[k], weight_size_list[k+1]]), name='W_gc_%d' % k)
            all_weights['b_gc_%d' %k] = tf.Variable(
                initializer([1, weight_size_list[k+1]]), name='b_gc_%d' % k)

            all_weights['W_bi_%d' % k] = tf.Variable(
                initializer([weight_size_list[k], weight_size_list[k + 1]]), name='W_bi_%d' % k)
            all_weights['b_bi_%d' % k] = tf.Variable(
                initializer([1, weight_size_list[k + 1]]), name='b_bi_%d' % k)

            #all_weights['W_mlp_%d' % k] = tf.Variable(
            #    initializer([weight_size_list[k], weight_size_list[k+1]]), name='W_mlp_%d' % k)
            #all_weights['b_mlp_%d' % k] = tf.Variable(
            #    initializer([1, weight_size_list[k+1]]), name='b_mlp_%d' % k)

        return all_weights

In [172]:
weights = _init_weights( emb_dim = emb_dim, weight_size = weight_size, n_layers = n_layers)

In [173]:
for cle, valeur in weights.items():
    print(cle, "shape : ",valeur.shape)

user_embedding shape :  (943, 40)
item_embedding shape :  (1682, 40)
W_gc_0 shape :  (40, 64)
b_gc_0 shape :  (1, 64)
W_bi_0 shape :  (40, 64)
b_bi_0 shape :  (1, 64)
W_gc_1 shape :  (64, 64)
b_gc_1 shape :  (1, 64)
W_bi_1 shape :  (64, 64)
b_bi_1 shape :  (1, 64)


## Accès aux données

In [174]:
# users, pos_items, neg_items = data_generator.sample()

# un neg_item c'est un item du train qui n'est pas dans le batch

data_generator = Data(path='../Data/ml-100k', batch_size=batch_size)



n_users=943, n_items=1682
n_interactions=100000
n_train=90404, n_test=9596, sparsity=0.06305


Pour générer un batch on exécute la méthode sample :

In [175]:
users, pos_items, neg_items = data_generator.sample()

In [176]:
#pos_items, users

On remarque que,marginalement, pour un batch donné **des items sont à la fois perçus comme positifs et négatifs** (faire tourner plusieurs fois la cellule data_generator.sample() si besoin)

In [177]:
len(list(set(pos_items) & set(neg_items)))

2

## Aperçu de la loss

**ATTENTION** ne sera pertinent qu'après avoir transformé les intrants en embedding

On constate qu'avec un argument négatif très fort la log sigmoid retourne - $\infty$  : 

In [179]:
def create_bpr_loss(decay, batch_size, users, pos_items, neg_items):
        pos_scores = tf.reduce_sum(tf.multiply(users, pos_items))
        neg_scores = tf.reduce_sum(tf.multiply(users, neg_items))

        regularizer = tf.nn.l2_loss(users) + tf.nn.l2_loss(pos_items) + tf.nn.l2_loss(neg_items)
        regularizer = regularizer/batch_size
        
        # In the first version, we implement the bpr loss via the following codes:
        # We report the performance in our paper using this implementation.
        maxi = tf.math.log(tf.nn.sigmoid(pos_scores - neg_scores))
        print("Maxi : ",maxi)
        mf_loss = tf.negative(tf.reduce_mean(maxi))
        print("MF loss V1 :",mf_loss)
        ## In the second version, we implement the bpr loss via the following codes to avoid 'NAN' loss during training:
        ## However, it will change the training performance and training performance.
        ## Please retrain the model and do a grid search for the best experimental setting.
        mf_loss2 = tf.reduce_sum(tf.nn.softplus(-(pos_scores - neg_scores)))
        print("MF loss V2 :",mf_loss2)

        emb_loss = decay * regularizer

        reg_loss = tf.constant(0.0, tf.float32, [1])

        return pos_scores, neg_scores,  mf_loss, emb_loss, reg_loss

In [180]:
users2     = tf.constant(users, dtype='float32')
pos_items2 = tf.constant(pos_items, dtype='float32')
neg_items2 = tf.constant(neg_items, dtype='float32')

tf.multiply(users2, pos_items2)
pos_scores = tf.reduce_sum(tf.multiply(users2, pos_items2))
neg_scores = tf.reduce_sum(tf.multiply(users2, neg_items2))
print(pos_scores - neg_scores)



tf.Tensor(-6451864.0, shape=(), dtype=float32)


In [181]:
create_bpr_loss(decay=decay, batch_size=batch_size, users=users2, pos_items=pos_items2, neg_items=neg_items2)

Maxi :  tf.Tensor(-inf, shape=(), dtype=float32)
MF loss V1 : tf.Tensor(inf, shape=(), dtype=float32)
MF loss V2 : tf.Tensor(6451864.0, shape=(), dtype=float32)


(<tf.Tensor: shape=(), dtype=float32, numpy=9452200.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=15904064.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=inf>,
 <tf.Tensor: shape=(), dtype=float32, numpy=68706.85>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>)

L'ordre de grandeur de la loss v2 peut devenir trop important et je garde en tête d'appliquer éventuellement un **clipping** car on peut avoir des risques d'explosion de gradient.

## Matrice d'adjacence

L'objet Data dans le module utiity/load_data contient l'outillage pour générer des matrices d'adjacence et les sauvegardée sur disque. Si les matrices ne sont pas déjà existentes elles sont crées alors dans le dossier Data du jeu de données en cours.

In [193]:
plain_adj, norm_adj, mean_adj = data_generator.get_adj_mat()

already load adj matrix (2625, 2625) 0.0281069278717041


3 matrices sont ainsi crées, on n'en utilise qu'une seule selon l'option précisée adj_type. J'ai une préférence pour une matrice normalisée.  

In [194]:
norm_adj

<2625x2625 sparse matrix of type '<class 'numpy.float64'>'
	with 183433 stored elements in Compressed Sparse Row format>

On regarde les valeurs de quelques éléments :

In [195]:
norm_adj[0,0],  norm_adj[25,30] , norm_adj[100,100]

(0.0040650406504065045, 0.0, 0.016129032258064516)

Pour la suite on a besoin de l'attribut n_nonzero_elems :


In [196]:
n_nonzero_elems = norm_adj.count_nonzero()
n_nonzero_elems

183433

# Création des embeddings

Le code gère la problématique de saturation RAM en splittant la matrice d'adjacence dans une liste de sous-matrice et en prenant en compte possiblement l'utilisation de drop out.

On a besoin de pas mal d'outillage pour créer les embeddings :

In [197]:
def _create_ngcf_embed(node_dropout_flag , norm_adj, weights, mess_dropout, n_users, n_items, n_layers, n_fold , node_dropout):
# Generate a set of adjacency sub-matrix.
    if node_dropout_flag:
        # node dropout.
        A_fold_hat = _split_A_hat_node_dropout(norm_adj)
    else:
        A_fold_hat = _split_A_hat(norm_adj)

    ego_embeddings = tf.concat([weights['user_embedding'], weights['item_embedding']], axis=0)

    all_embeddings = [ego_embeddings]

    for k in range(0, n_layers):

        temp_embed = []
        for f in range(n_fold):
            temp_embed.append(tf.sparse.sparse_dense_matmul(A_fold_hat[f], ego_embeddings))

        # sum messages of neighbors.
        side_embeddings = tf.concat(temp_embed, 0)
        print('shape side_embeddings : ',side_embeddings.shape)
        
        # transformed sum messages of neighbors.
        sum_embeddings = tf.nn.leaky_relu( tf.matmul(side_embeddings, weights['W_gc_%d' % k]) + weights['b_gc_%d' % k] )
        print('shape sum_embeddings : ', sum_embeddings.shape)

        # bi messages of neighbors.
        bi_embeddings = tf.multiply(ego_embeddings, side_embeddings)
        print('shape bi_embeddings : ', bi_embeddings.shape)
        # transformed bi messages of neighbors.
        bi_embeddings = tf.nn.leaky_relu( tf.matmul(bi_embeddings, weights['W_bi_%d' % k]) + weights['b_bi_%d' % k] )
        print('shape bi_embeddings transformed : ', bi_embeddings.shape)
        
        # non-linear activation.
        ego_embeddings = sum_embeddings + bi_embeddings
        print('shape ego embeddings : ', ego_embeddings.shape)

        # message dropout.
        ego_embeddings = tf.nn.dropout(ego_embeddings, 1 - mess_dropout[k])

        # normalize the distribution of embeddings.
        norm_embeddings = tf.nn.l2_normalize(ego_embeddings, axis=1)

        all_embeddings += [norm_embeddings]
        print('****** fin boucle for *****')

    all_embeddings = tf.concat(all_embeddings, 1)
    print('shape all_embeddings : ', all_embeddings.shape)
    u_g_embeddings, i_g_embeddings = tf.split(all_embeddings, [n_users, n_items], 0)
    print('shape u_g_embeddings et i_g : ', u_g_embeddings.shape, i_g_embeddings.shape)
    return u_g_embeddings, i_g_embeddings



def _convert_sp_mat_to_sp_tensor(X):
    coo = X.tocoo().astype(np.float32)
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

def _dropout_sparse(X, keep_prob, n_nonzero_elems):
    """
    Dropout for sparse tensors.
    """
    noise_shape = [n_nonzero_elems]
    random_tensor = keep_prob
    random_tensor += tf.random.uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse.retain(X, dropout_mask)

    return pre_out * tf.math.divide(1., keep_prob)



def _split_A_hat(X):
    A_fold_hat = []

    fold_len = (n_users + n_items) // n_fold
    for i_fold in range(n_fold):
        start = i_fold * fold_len
        if i_fold == n_fold -1:
            end = n_users + n_items
        else:
            end = (i_fold + 1) * fold_len

        A_fold_hat.append(_convert_sp_mat_to_sp_tensor(X[start:end]))
    return A_fold_hat

def _split_A_hat_node_dropout( X):
    A_fold_hat = []

    fold_len = (n_users + n_items) // n_fold
    for i_fold in range(n_fold):
        start = i_fold * fold_len
        if i_fold == n_fold -1:
            end = n_users + n_items
        else:
            end = (i_fold + 1) * fold_len

        # A_fold_hat.append(self._convert_sp_mat_to_sp_tensor(X[start:end]))
        temp = _convert_sp_mat_to_sp_tensor(X[start:end])
        n_nonzero_temp = X[start:end].count_nonzero()
        A_fold_hat.append(_dropout_sparse(temp, 1 - node_dropout[0], n_nonzero_temp))

    return A_fold_hat



In [198]:
ua_embeddings, ia_embeddings = _create_ngcf_embed(node_dropout_flag=1 , norm_adj = norm_adj , weights = weights , node_dropout = node_dropout, mess_dropout = mess_dropout, n_users = n_users , n_items = n_items,n_layers = n_layers, n_fold = 100)

shape side_embeddings :  (2625, 40)
shape sum_embeddings :  (2625, 64)
shape bi_embeddings :  (2625, 40)
shape bi_embeddings transformed :  (2625, 64)
shape ego embeddings :  (2625, 64)
****** fin boucle for *****
shape side_embeddings :  (2625, 64)
shape sum_embeddings :  (2625, 64)
shape bi_embeddings :  (2625, 64)
shape bi_embeddings transformed :  (2625, 64)
shape ego embeddings :  (2625, 64)
****** fin boucle for *****
shape all_embeddings :  (2625, 168)
shape u_g_embeddings et i_g :  (943, 168) (1682, 168)


**Commentaires**

On note le passage entre ego_embeddings et all_ebeddings :


    L'opérateur += va ajouter le nb de colonnes en concatenant chaque embeddings d'une couche.
    
    Deux layers à 64 units donne une sortie à 128, trois layers donne 64*3

Les premières lignes de all_ebeddings sont pour les users et les dernièrs pour les items.

**Dimensions de sortie** 

Pour emb_dim = 40 et 2 layers de 64 on obtient **40 + 64 + 64 = 168 colonnes**

On peut maintenant obtenir les coordonnées des users, des items positifs et négatif pour notre batch en cours dans notre espace :

In [199]:
#  Establish the final representations for user-item pairs in batch.
u_g_embeddings = tf.nn.embedding_lookup(ua_embeddings    , users)
pos_i_g_embeddings = tf.nn.embedding_lookup(ia_embeddings, pos_items)
neg_i_g_embeddings = tf.nn.embedding_lookup(ia_embeddings, neg_items)

In [200]:
u_g_embeddings.shape, pos_i_g_embeddings.shape, neg_i_g_embeddings.shape

(TensorShape([50, 168]), TensorShape([50, 168]), TensorShape([50, 168]))

On peut mainenant réaliser une inférence (ce sera utile en testing):

In [201]:
batch_ratings = tf.matmul(u_g_embeddings, pos_i_g_embeddings, transpose_a=False, transpose_b=True)
batch_ratings.shape

TensorShape([50, 50])

In [202]:
_, _, mf_loss, emb_loss, reg_loss = create_bpr_loss(decay=decay, batch_size=batch_size, users=u_g_embeddings, pos_items=pos_i_g_embeddings, neg_items=neg_i_g_embeddings )
loss = mf_loss + emb_loss + reg_loss
print(mf_loss,emb_loss,reg_loss, loss)

Maxi :  tf.Tensor(-0.49957103, shape=(), dtype=float32)
MF loss V1 : tf.Tensor(0.49957103, shape=(), dtype=float32)
MF loss V2 : tf.Tensor(0.49957103, shape=(), dtype=float32)
tf.Tensor(0.49957103, shape=(), dtype=float32) tf.Tensor(0.3084888, shape=(), dtype=float32) tf.Tensor([0.], shape=(1,), dtype=float32) tf.Tensor([0.8080598], shape=(1,), dtype=float32)
